## preparation

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize
import random

In [2]:
fares = pd.read_csv('Fares.csv', sep = ';')
flights = pd.read_csv('Flights.csv', sep = ';')


In [3]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [4]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
history['id'] = '001'
for i in range(2,101,1):
    if len(str(i)) == 1:
        i = '00' + str(i)
    elif len(str(i)) == 2:
        i = '0' + str(i)
    else:
        i = str(i)
        
        
    hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_' + i + '.csv', sep = ';')
    hist['id'] = i
    history = history.append(hist)
    
    
   

In [121]:
def get_demands(fare_id, history):
    filt_history = history.loc[history.FARE_ID == fare_id,]
    demands = filt_history['id'].value_counts()
    return demands

def get_capacity(flight_id):
    return flights.loc[flights.FLIGHT_ID == flight_id]['CAPACITY']
    
def get_price(fare_id):
    return fares.loc[fares.FARE_ID == fare_id]['PRICE']

def get_flights(fare_id):
    dep_air  = int(fares.loc[fares.FARE_ID == fare_id]['DEPARTURE_AIRPORT'])
    dest_air = int(fares.loc[fares.FARE_ID == fare_id]['DESTINATION_AIRPORT'])
   
    if dep_air == 1 or dest_air == 1:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air) &
                           (flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    else:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air)]['FLIGHT_ID'], 
                flights.loc[(flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    
def get_shadow_price(fare_id, shadow_prices_df):
    flights_needed = get_flights(fare_id)
    if len(flights_needed) == 1:
        shadow_price = shadow_prices_df[flights_needed[0] -1]
    else:
        shadow_price = shadow_prices_df[flights_needed[0]-1] + shadow_prices_df[flights_needed[1]-1]
        
    return(shadow_price)
    
    
def allocate_seat(allocations, shadow_prices_df, fare_id, sold_tickets, flight_fare_df, bucket_flights):
    if allocations[fare_id-1] > 0:
        allocations[fare_id-1] -= 1
        sold_tickets[fare_id-1] += 1 
    else :
        shadow_price = get_shadow_price(fare_id, shadow_prices_df)
        flights_needed = get_flights(fare_id)
#         flights_needed = get_flights(fare_id)
#         if len(flights_needed) == 1:
#             shadow_price = shadow_prices_df[flights_needed-1]
#         else:
#             shadow_price = shadow_prices_df[flights_needed[0]-1] + shadow_prices_df[flights_needed[1]-1]
      
        if len(flights_needed) == 1:
            if flights_needed in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[flights_needed[0] - 1,] == 1)[0]
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                                                
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i]+1, shadow_prices_df)
                
                sacrificed_fare = \
                candidate_fares[np.where(candidate_fare_shadow_prices == \
                                         candidate_fare_shadow_prices.min())[0][0]]
                
                allocations[sacrificed_fare-1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) +1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
                
        else:
            if flights_needed in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[flights_needed[0] - 1,] == 1)[0]
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i] +1, shadow_prices_df)
                
                sacrificed_fare = \
                candidate_fares[np.where(candidate_fare_shadow_prices == \
                                         candidate_fare_shadow_prices.min())[0][0]]
                
                allocations[sacrificed_fare-1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) + 1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
                
            if flights_needed[1] in bucket_flights:
                bucket_flights = np.delete(bucket_flights,np.where(bucket_flights == flights_needed)[0])
                sold_tickets[fare_id-1] +=1
            else:
                candidate_fares = np.where(flight_fare_df.iloc[flights_needed[0] - 1,] == 1)[0]
                fully_booked_fares = np.array(np.where(allocations == 0)) + 1
                candidate_fares = candidate_fares[~np.in1d(candidate_fares,fully_booked_fares)]
                
                candidate_fare_shadow_prices = np.zeros((candidate_fares.shape))
                for i in range(candidate_fares.shape[0]):
                    candidate_fare_shadow_prices[i] = get_shadow_price(candidate_fares[i] +1, shadow_prices_df)
                
                sacrificed_fare = \
                candidate_fares[np.where(candidate_fare_shadow_prices == \
                                         candidate_fare_shadow_prices.min())[0][0]]
                
                allocations[sacrificed_fare - 1] -= 1
                sold_tickets[fare_id-1] += 1
                
                if sum(flight_fare_df.iloc[:,sacrificed_fare-1]) == 2:
                    sf_flights = np.array(np.where(flight_fare_df.iloc[:,sacrificed_fare-1] == 1)) + 1
                    sf_flights = np.delete(sf_flights, sf_flights == flights_needed)
                    bucket_flights = np.append(bucket_flights, sf_flights)
            
            #napocitat ktery fares maj volny alokace, porovnat shadow pricy, vzit jim je, priradit a zbytek do bucketu
            
        
        
#     return allocations, shadow_prices_df, fare_id, sold_tickets, bucket_flights  
    
def run_allocations(history_day, allocations, shadow_prices_calc, flight_fare_df):
    sold_tickets = np.zeros((144,1))
    bucket_flights = np.array([])
    for i in range(history_day.shape[0]):
        allocate_seat(allocations, shadow_prices_calc, history_day['FARE_ID'][i], sold_tickets, flight_fare_df, bucket_flights)
        print(i)
    
    return sold_tickets
    




    

In [110]:
flights_needed = get_flights(1)
flights_needed in np.array([])

C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


False

In [35]:
shadow_prices = scipy.optimize.linprog(c=capacity.reshape((16,)), A_ub= - np.array(flight_fare_df).T,
                      b_ub= - np.array(fares['PRICE']).T).x

C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [122]:
hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
tickets = run_allocations(hist, allocations, shadow_prices, flight_fare_df)

C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: FutureWarning: in the future insert will treat boolean arrays and array-likes as boolean index instead of casting it to integer
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: in the future insert will treat boolean arrays and array-likes as b

0
1
2
3


C:\Users\Radim\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251


In [130]:
tickets

array([[ 3.],
       [ 5.],
       [ 3.],
       [ 0.],
       [ 5.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 3.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 4.],
       [ 0.],
       [ 4.],
       [ 3.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 6.],
       [ 6.],
       [ 2.],
       [ 2.],
       [ 4.],
       [ 2.],
       [12.],
       [10.],
       [ 2.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 6.],
       [ 3.],
       [ 6.],
       [ 4.],
       [ 2.],
       [ 2.],
       [ 6.],
       [ 2.],
       [ 2.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 6.],
       [ 2.],
       [12.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 6.],
       [ 6.],
       [ 6.],
       [ 0.],
       [10.],
       [ 6.],
       [ 2.],
       [ 2.],
       [ 4.],
       [ 0.],
       [12.],
       [ 6.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 2.],
       [ 0.],
       [ 6.],
       [ 8.],
       [ 4.],
      

In [124]:
fares

,FARE_ID,DEPARTURE_AIRPORT,DESTINATION_AIRPORT,CLASS,PRICE
0,1,1,2,1,24
1,2,1,2,2,192
2,3,1,3,1,34
3,4,1,3,2,272
4,5,1,4,1,67
5,6,1,4,2,536
6,7,1,5,1,62
7,8,1,5,2,496
8,9,1,6,1,61
9,10,1,6,2,488


In [92]:
a = np.array([1, 2, 3, 5])

b = np.array([4, 4, 2, 3, 1, 0])

print(np.in1d(b,a))

print(b[~np.in1d(b,a)])

[False False  True  True  True False]
[4 4 0]


In [79]:
for i in range(0):
    print(i)

In [25]:
flight_fare_df = pd.DataFrame(index=range(flights.shape[0]),columns=range(fares.shape[0]), data = 0)
for i in range(fares.shape[0]):
    used_flights = get_flights(i+1)
    try: 
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        flight_fare_df.iloc[int(used_flights[1]) - 1, i] = 1
    except:
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        

## optimization

In [26]:
capacity = flights['CAPACITY']
np.reshape(capacity, (16)).shape
exp_demand = np.array(history['FARE_ID'].value_counts())
bounds = [(0, exp_demand[i]) for i in range(len(exp_demand))]
print(exp_demand.shape)

x = scipy.optimize.linprog(c= - np.array(fares['PRICE']), A_ub=np.array(flight_fare_df),
                      b_ub=capacity, bounds=bounds)

(144,)


C:\Users\Radim\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [28]:
allocations = x.x

In [16]:
np.sum(x.x)


182.0

In [312]:
(np.reshape(np.array(flight_fare_df) @ x0,(16))).shape

(16,)

array([-0.])

In [306]:
(np.array(flight_fare_df) @ np.zeros((fares.shape[0],1))).shape

(16, 1)

19380530